# Analyse des données du PIM

## Extraction des données

### Préambule technique

In [1]:
# setting up sys.path for relative imports
from pathlib import Path
import sys
project_root = str(Path(sys.path[0]).parents[1].absolute())
if project_root not in sys.path:
    sys.path.append(project_root)

In [68]:
# imports and customization of diplay
import pandas as pd
pd.options.display.min_rows = 6
pd.options.display.width=108
import numpy as np

from src.pimapi import Requester

### Récupération des données

Le requêtage des données dans le PIM s'appuie sur la classe `Requester` du module pimapi.

In [3]:
requester = Requester('prd')
# Let's fetch the full content of PIM system
requester.fetch_all_from_PIM()
requester.result

Done


[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

A ce stade, les données sont chargées en mémoire sous la forme de fichiers JSON. La conversion des données récupérées par l'API se fait via la méthode `result_to_dataframe` de la classe `Requester`.

In [5]:
df = requester.result_to_dataframe()
df.head(4)

,entity-type,repository,path,type,state,parentRef,isCheckedOut,isVersion,isProxy,changeToken,...,properties.pprodqmdd:manufacturingDiagram.length,properties.pprodqmdd:manufacturingDiagram.data,properties.pprodq:visualPhoto.name,properties.pprodq:visualPhoto.mime-type,properties.pprodq:visualPhoto.encoding,properties.pprodq:visualPhoto.digestAlgorithm,properties.pprodq:visualPhoto.digest,properties.pprodq:visualPhoto.length,properties.pprodq:visualPhoto.data,properties.notif:notifications
uid,,,,,,,,,,,,,,,,,,,,,
afee12c7-177e-4a68-9539-8cbb68442503,document,default,/default-domain/pomSupplierWorkspace/SICO/DEST...,pomProduct,product.waiting.supplier.validation,a58845c0-cab3-492f-b48d-531f146c3777,True,False,False,17-0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7d390121-17e8-43bf-a357-9d06b79d2d47,document,default,/default-domain/pomSupplierWorkspace/UNILEVER_...,pomProduct,product.waiting.supplier.validation,a37abc27-f485-4ae9-921b-f761f16c8c1c,False,False,False,15-0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f234cd84-c8f6-433f-85ec-6e0b6980adc6,document,default,/default-domain/pomSupplierWorkspace/AZTECA_FO...,pomProduct,product.waiting.supplier.validation,3ff7819a-a392-493f-beb8-0b323ac331c7,True,False,False,33-0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e82a8173-b379-41ac-b319-aa058a04fcfb,document,default,/default-domain/pomSupplierWorkspace/UVCDR_-_C...,pomProduct,product.waiting.sending.supplier,e4b5167c-ece2-4f7a-83c1-fb884034a1bf,False,False,False,19-0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Définition des libellés pour les mises en formes

In [70]:
lab = {
    'code': 'Code produit', 
    'supplier': 'Code fournisseur',
    'type': 'Type de produit',
    'GTIN': 'GTIN',
    'base_unit': 'Unité de base', 
    'net_weight': 'Poids net',
    'gross_weight': 'Poids brut', 
    'dry_weight': 'Poids net égoutté',
    'volume': 'Volume',
}

## Description des attributs des produit

### Volumétrie

On constate que chaque produit porte un très grand nombre d'attributs : 

In [6]:
print('Count of columns in df:', len(df.columns))
print('\nInfo of df:')
df.info()

Count of columns in df: 487

Info of df:
<class 'pandas.core.frame.DataFrame'>
Index: 13179 entries, afee12c7-177e-4a68-9539-8cbb68442503 to 6dfce29e-fd4c-4670-9f9c-5c02a5b4d52a
Columns: 487 entries, entity-type to properties.notif:notifications
dtypes: bool(12), float64(60), int64(2), object(413)
memory usage: 48.0+ MB


De plus, de par la nature hiérarchique du format JSON, certains attributs dits "multivalués" sont parfois stockés sous forme de liste dans le dataframe "à plat".
Par exemple, on peut voir que le pays de transformation, ou les facettes, peuvent être multivalués.

In [8]:
df.loc[['609af223-2f14-4f83-a553-cef276f2eca7',
        'c94013e4-0dca-441a-85c1-0b29ecb54d0a',
        '82d1af25-2bdd-4315-9670-67784b70dfa7'],
       ['properties.pprodg:transfoCountries',
        'facets']]

,properties.pprodg:transfoCountries,facets
uid,,
609af223-2f14-4f83-a553-cef276f2eca7,"[PL, FR, ES]","[Versionable, Folderish, Commentable, beginnin..."
c94013e4-0dca-441a-85c1-0b29ecb54d0a,"[DE, NO, BE, RU, CH, BG, LT, GR, FR, UA, HU, E...","[endMigration, Versionable, Folderish, Comment..."
82d1af25-2bdd-4315-9670-67784b70dfa7,[FR],"[endMigration, Versionable, Folderish, Comment..."


De plus, certains attributs sont dits "complexes", car chacune des valeurs de la liste est elle-même un dictionnaire d'attribut.
La combinaison des deux, des attributs "complexes multivalués" existe également. On a alors une liste de dictionnaires. On peut comme ceci imbriquer des niveaux jusqu'à n'importe quelle profondeur.

C'est par exemple le cas des labels qui sont multivalués (un produit peut porter plusieurs labels), qui sont des complexes portant :
- le type de label (bio, Label Rouge, ...)
- la date de fin de validité du label (si applicable)
- le fichier de certification du label (si applicable), qui est lui-même un complexe...


In [9]:
multilabel_ds = df.loc[df['properties.pprodl:labels'].apply(len) > 1, 'properties.pprodl:labels']
for uid, label_list in multilabel_ds.head(3).iteritems():
    print('product uid:', uid)
    for cpt, label in enumerate(label_list):
        print('\n\tlabel', cpt + 1, ':')
        for key, val in label.items():
            print('\t\t', key, ':', val)
    print('-------------------------------------')    

product uid: 3c2a8d1a-634d-40bb-9852-81eb8a340114

	label 1 :
		 labelCertificateEndDate : None
		 typeOfLabel : 30
		 labelCertificateFile : None

	label 2 :
		 labelCertificateEndDate : None
		 typeOfLabel : 40
		 labelCertificateFile : None
-------------------------------------
product uid: d3681e26-b024-4603-ae0b-0d5630329fa0

	label 1 :
		 labelCertificateEndDate : 2020-03-30T22:00:00.000Z
		 typeOfLabel : 100
		 labelCertificateFile : {'name': 'SAS - Certificat AB V2.pdf', 'mime-type': 'application/pdf', 'encoding': None, 'digestAlgorithm': 'MD5', 'digest': '1d424d2d2c9539abca07b8ad9576a339', 'length': '128780', 'data': 'https://produits.groupe-pomona.fr/nuxeo/nxfile/default/d3681e26-b024-4603-ae0b-0d5630329fa0/pprodl:labels/0/labelCertificateFile/SAS%20-%20Certificat%20AB%20V2.pdf?changeToken=92-0'}

	label 2 :
		 labelCertificateEndDate : 2020-03-30T22:00:00.000Z
		 typeOfLabel : 80
		 labelCertificateFile : {'name': 'SAS - Certificat AB V2.pdf', 'mime-type': 'application/pdf',

In [65]:
df.describe()

,properties.pprodc:degreeOfAlcohol,properties.pprodqmdd:primaryPackagingPhoto,properties.pprodqmdd:mddPicture,properties.pprodqmdd:controlPlan,properties.pprodqmdd:secondaryPackagingPhoto,properties.pprodqmdd:manufacturingDiagram,properties.pprodp:paletteLength,properties.pprodp:uqbByColis,properties.pprodp:paletteGrossWeight,properties.pprodp:paletteHeight,...,properties.thumb:thumbnail.encoding,properties.pprodcd:certificate.encoding,properties.thumb:thumbnail,properties.pprodad:securityForm.encoding,properties.pprodqmdd:primaryPackagingPhoto.encoding,properties.pprodqmdd:mddPicture.encoding,properties.pprodqmdd:controlPlan.encoding,properties.pprodqmdd:secondaryPackagingPhoto.encoding,properties.pprodqmdd:manufacturingDiagram.encoding,properties.pprodq:visualPhoto.encoding
count,581.000000,0.0,0.0,0.0,0.0,0.0,7892.000000,12857.000000,5475.000000,7.895000e+03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,12.673081,NaN,NaN,NaN,NaN,NaN,1182.484161,10.289881,781.985262,3.733925e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,7.994323,NaN,NaN,NaN,NaN,NaN,1400.230698,36.341406,13956.084798,2.026316e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,11.000000,NaN,NaN,NaN,NaN,NaN,1200.000000,1.000000,252.614000,1.200000e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,12.500000,NaN,NaN,NaN,NaN,NaN,1200.000000,6.000000,512.590000,1.426000e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,13.000000,NaN,NaN,NaN,NaN,NaN,1200.000000,10.000000,763.000000,1.650000e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,60.000000,NaN,NaN,NaN,NaN,NaN,120000.000000,2000.000000,730600.000000,1.800556e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
mappings = {
    'identification': {
      'properties.vig:code': 'code',
      'properties.psec:supplierCode': 'supplier',
      'properties.pprodtop:typeOfProduct': 'type',
      'properties.pprodi:gtin': 'GTIN',
    },
    'dimensions': {
        'properties.pprodtop:baseUnit': 'base_unit',
        'properties.pprodg:netWeight': 'net_weight',
        'properties.pprodg:grossWeight': 'gross_weight',
        'properties.pprodg:dryWeight': 'dry_weight',
        'properties.pprodg:volume': 'volume',
    },
   
  }

# TODO !!! Faire les bons arrdondis pour l'export en latex des tables. Trop de zéros non significatifs !


for map_type, mapping in mappings.items():
    cur_df = df.loc[:, list(mapping.keys())].rename(mapping, axis=1).fillna(np.nan)
    desc = cur_df.describe(include='all')
    samp = cur_df.sample(n=5, random_state=42)
    print(map_type)
    print(samp.rename(lab, axis=1))
    print('---------------------------------------------------')
    print(desc.rename(lab, axis=1))
    print('---------------------------------------------------')
    c_format = 'l' + 'c' * len(cur_df.columns)
    (samp.rename(lab, axis=1)
         .to_latex(Path('..') / 'tbls' / ('Exemple '+ map_type +'.tex'),
                   bold_rows=True,
                   column_format=c_format,
                   na_rep='-'
                   ))
    (desc.rename(lab, axis=1)
         .to_latex(Path('..') / 'tbls' / ('Desc '+ map_type +'.tex'),
                   bold_rows=True,
                   column_format=c_format,
                   na_rep='-'
                   ))

identification
                                         Code produit Code fournisseur Type de produit            GTIN
uid                                                                                                   
1c6a9d67-8fea-4b5e-a0c3-29f9338e1128  PIMP-0000004399  PIMF-0000000112  alcoholicDrink   3080210001100
07d045b6-4cde-403d-b577-a11b899dcd29  PIMP-0000005503  PIMF-0000000060         grocery   3760128846009
40b933d8-d8eb-4867-8d14-83fc999c5281  PIMP-0000010837  PIMF-0000000011         grocery  13274643110097
ae72ae63-7b12-4bcf-b15e-52acff941375  PIMP-0000003913  PIMF-0000000283         hygiene   3342690094301
81ed6169-03eb-44b5-a3eb-8a25e879948e  PIMP-0000010172  PIMF-0000000178         hygiene             NaN
---------------------------------------------------
           Code produit Code fournisseur Type de produit   GTIN
count             13179            13179           13179  12015
unique            13179              605               5  11329
top     PIMP-0000000